In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime 
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
#filename = 'coinmktcap-ticker.json'
url = 'https://api.coinmarketcap.com/v1/ticker/?limit=100'
colors = ['rgb(204,0,0)', 'rgb(38,77,0)', 'rgb(38,77,0)']
day_vol_threshold = 5000  # Only want trading vol >= $5K
#df = pd.read_json('../data/{}'.format(filename))
df = pd.read_json(url)
df = df[df['24h_volume_usd'].ge(day_vol_threshold)]
layout = go.Layout(
    title='Cryptocurrency Daily Turnover Rate (@ Exchanges)<br />As-of {:%B %-d, %Y %H:%M:%S} UTC'.format(
        datetime.utcfromtimestamp(df[df['id'] == 'bitcoin'].iloc[0]['last_updated'])),
    xaxis=dict(
        title='24H Turnover Ratio',
        type='log',
        ticksuffix='%',
        dtick=0.5,
    ),
    yaxis=dict(
        title='Market Cap',
        type='log',
    ),
    hovermode = 'closest',
)
data = [
    go.Scatter(
        mode='markers',
        hoverinfo='text',
        x=(df['24h_volume_usd'] / df['market_cap_usd']) * 100,
        y=df['market_cap_usd'],
        text=df['name'] + ' (' + df['symbol'] + ')<br />24 Hour: ' + df['percent_change_24h'].astype('str') + '%<br />7 Day: ' + df['percent_change_7d'].astype('str') + '%',
        marker=dict(
            color=[colors[int(np.sign(x)) + 1] for x in df['percent_change_24h']],
            size=np.log(df['24h_volume_usd']),
        ),
    )]
fig = go.Figure(data=data, layout=layout)
iplot(fig)